In [1]:
!pip install boto3
!pip install plotly

In [7]:
# Import librairies
import pandas as pd
import glob
import boto3

In [8]:
# Import dataset
data = pd.read_csv('results_cities_to_visit.csv')
data.head()

,ville_id,nom,latitude,longitude,meteo,proba_pluie,jour_temperature,rang
0,20,Avignon,43.949249,4.805901,Clouds,0.5957,26.05,1
1,22,Nimes,43.837425,4.360069,Clouds,2.6465,26.17,2
2,15,Lyon,45.757814,4.832011,Rain,2.9370,19.72,3
3,21,Uzes,44.012128,4.419672,Clouds,3.5270,25.75,4
4,24,Saintes Maries de la mer,43.452277,4.428717,Clouds,3.6753,22.72,5


In [9]:
# # We recover with the .glob () function, the files which start with Hotels and which end with the .json format
hotels_results = glob.glob('Hotels_*.json')
hotels_results

['Hotels_Paris.json',
 'Hotels_Dijon.json',
 'Hotels_Montauban.json',
 'Hotels_Marseille.json',
 'Hotels_Amiens.json',
 'Hotels_Avignon.json',
 'Hotels_Strasbourg.json',
 'Hotels_Collioure.json',
 'Hotels_Carcassonne.json',
 'Hotels_Nimes.json',
 'Hotels_Colmar.json',
 'Hotels_Besancon.json',
 'Hotels_Grenoble.json',
 'Hotels_Biarritz.json',
 'Hotels_St-Malo.json',
 'Hotels_Annecy.json',
 'Hotels_Aigues-Mortes.json',
 'Hotels_Le-Havre.json',
 'Hotels_Bayonne.json',
 'Hotels_La-Rochelle.json',
 'Hotels_Saintes-Maries-de-la-mer.json',
 'Hotels_Toulouse.json',
 'Hotels_Cassis.json',
 'Hotels_Lille.json',
 'Hotels_Bormes-les-Mimosas.json',
 'Hotels_Eguisheim.json',
 'Hotels_Uzes.json',
 'Hotels_Aix-en-Provence.json',
 'Hotels_Lyon.json',
 'Hotels_Rouen.json',
 'Hotels_Bayeux.json']

In [10]:
# Create a new dataframe
cols= ['ville_id', 'ville_nom', 'hotel_id', 'url', 'latitude', 'longitude', 'score', 'description']
hotels = pd.DataFrame(columns = cols)
hotels

,ville_id,ville_nom,hotel_id,url,latitude,longitude,score,description


In [11]:
# Iteration on all .JSON files
for h in hotels_results:
    ville_nom = h.split('_')[1].split('.')[0].replace("-"," ") # Data separation with "_" and with ".", we replace "_" by a space
    ville_id = data.loc[data['nom'] == ville_nom,'ville_id'].values[0]
     
    # Reading .json  file(s) and  ville_id, hotel_id and ville_nom into the new dataframe
    df = pd.read_json(h)
    df = df.reset_index().rename({'index': 'hotel_id'}, axis = 1)
    df.loc[:,'ville_id'] = ville_id
    df.loc[:,'ville_nom'] = ville_nom
    
    #Cleaning text
    df.loc[:, 'name'] = df['name'].str.replace('\n', '') # We replace the symbol \ n by spaces for the column "nom"
    df.loc[:, 'url'] = df['url'].str.replace('\n', '') # We replace the symbol \ n by spaces for the column  "url"
    df.loc[:, 'description'] = df['description'].str.replace('\n', '') # We replace the symbol \ n by spaces for the column "description"
    
    # Extracting latitude and longitude
    df.loc[:,'coords'] = df['coords'].str.split(',') # Separation of coordinates with a comma
    df.loc[:, 'longitude'] = df['coords'].apply(lambda x : x[0]) # Assigning longitude to index 0
    df.loc[:, 'latitude'] = df['coords'].apply(lambda x : x[1]) # Assigning latitude to index 1
    
    # Removale "coords" column with drop() function (axis=1 is for column, axis=0 is for row)
    df = df.drop('coords', axis=1)
    
    # Add into the dataframe "hotels"
    hotels = hotels.append(df)

In [12]:
hotels.loc[:, 'score'] = hotels['score'].replace(',','.') # Replacement of commas by dots in the "score" column


In [13]:
# Sauvegarde du fichier "hotels" en fichier .csv
hotels.to_csv('Hotels2.csv', index=False)